## Create PAS ClaimResponse instances for testing and use by implementers

 - Use jinja2 templates to create a yaml template representing a PAS ClaimResponse instances
 - store instance data in a CSV file
   - columns equal the SD id ??
   - get CSV
   - Convert CDV to a DF
   - for each row generate dictionary substituting default values where missing
   - create a resource instance using the templates
   - convert to json
   - validate using bash
   - save

In [12]:
from jinja2 import Template, Environment, FileSystemLoader
import pandas as pd
import json
import uuid
import yaml
import pathlib

temp = pathlib.Path('validation/temp.json')
print(temp.resolve())
!pwd

/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/CDex_Demo/validation/temp.json
/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/CDex_Demo


In [4]:
# Load the environment and specify the folder containing templates
env = Environment(loader=FileSystemLoader('templates'))

# Load the template
template = env.get_template('pended_CRB_yaml.j2')

In [5]:
# Read CSV file with Pandas
csv_file = 'source_data/crb_test_data.csv'  
df = pd.read_csv(csv_file)
df

,bundle_id,trace_number,claim_type,created_date,patient_member_number,patient_family_name,patient_given_name
0,bundle-001,TRACE00001,professional,2025-02-21T12:00:00-08:00,M00001,Doe,Jane
1,bundle-002,TRACE00002,institutional,2025-02-20T10:00:00-08:00,M00002,Smith,John


In [22]:
# for each row generate dictionary substituting default values where missing

# Function to convert row to dictionary, generate UUIDs, and handle missing values
def row_to_data(row):
    print(row.name)
    bundle_uuid = str(uuid.uuid4())
    claim_response_uuid = str(uuid.uuid4())
    patient_uuid = str(uuid.uuid4())
    insurer_uuid = str(uuid.uuid4())
    requestor_uuid = str(uuid.uuid4())
    claim_uuid = str(uuid.uuid4())

    return {
        "bundle_id": row.get('bundle_id', bundle_uuid),
        "timestamp": row.get('timestamp', "2025-02-21T13:00:00-08:00"),
        "claim_response_uuid": claim_response_uuid,
        "trace_number": row.get('trace_number', f"TRACE{row.name:05d}"),
        "claim_type": row.get('claim_type', "professional"),
        "patient_uuid": patient_uuid,
        "created_date": row.get('created_date', "2025-02-21T12:00:00-08:00"),
        "insurer_uuid": insurer_uuid,
        "requestor_uuid": requestor_uuid,
        "claim_uuid": claim_uuid,
        "preauth_start_date": row.get('preauth_start_date', "2025-02-21"),
        "preauth_end_date": row.get('preauth_end_date', "2025-03-21"),
        "include_patient": row.get('include_patient', True),
        "patient_member_number": row.get('patient_member_number', f"M{row.name:05d}"),
        "patient_family_name": row.get('patient_family_name', "Smith"),
        "patient_given_name": row.get('patient_given_name', "John"),
        "include_insurer": row.get('include_insurer', True),
        "insurer_name": row.get('insurer_name', "Example Insurance Co"),
        "include_requestor": row.get('include_requestor', True),
        "requestor_name": row.get('requestor_name', "Example Clinic")
    }


# Process each row in the DataFrame
for index, row in df.iterrows():
    # Convert row to dictionary with UUIDs and defaults
    data = row_to_data(row)

# Render the template
    rendered = template.render(**data)
    print(rendered)

# convert to json save to temporary file
    rendered_dict = yaml.load(rendered, Loader=yaml.SafeLoader)
    rendered_json = json.dumps(rendered_dict, indent=2)
    # print(rendered_json)
    temp.write_text(rendered_json)
 
# validate using bash
    VALIDATOR ='/Users/ehaas/Downloads/validator_cli.jar' 
    RESOURCE = temp.resolve()
    VERSION = '4.0.1'
    IG_PACKAGE = 'hl7.fhir.us.davinci-pas'
    IG_VERSION  = '2.1.0'
    IG = f'{IG_PACKAGE}#{IG_VERSION}'
    print(IG)
    PROFILE = 'http://hl7.org/fhir/us/davinci-pas/StructureDefinition/profile-claimresponse'
    !echo "java -Xmx4G -jar $VALIDATOR $RESOURCE -version $VERSION  -ig $IG  -profile $PROFILE"
    !java -Xmx4G -jar $VALIDATOR $RESOURCE -version $VERSION  -ig $IG  -profile $PROFILE




0
resourceType: Bundle
id: "bundle-001"
meta:
  profile:
    - http://hl7.org/fhir/us/davinci-pas/StructureDefinition/profile-pas-response-bundle
type: collection
timestamp: "2025-02-21T13:00:00-08:00"
entry:
  - fullUrl: "urn:uuid:0b572b35-b8e9-478c-be27-6a1526cd6ef0"
    resource:
      resourceType: ClaimResponse
      id: "0b572b35-b8e9-478c-be27-6a1526cd6ef0"
      meta:
        profile:
          - http://hl7.org/fhir/us/davinci-pas/StructureDefinition/profile-claimresponse
      identifier:
        - system: http://example.org/PATIENT_EVENT_TRACE_NUMBER
          value: "TRACE00001"
      status: active
      type:
        coding:
          - system: http://terminology.hl7.org/CodeSystem/claim-type
            code: "professional"
      use: preauthorization
      patient:
        reference: "urn:uuid:4197a623-06d9-46b3-bb13-7a0674b899af"
      created: "2025-02-21T12:00:00-08:00"
      insurer:
        reference: "urn:uuid:eaa81da0-1ecf-4327-8f46-20d125732c53"
      requestor:
